In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import os

In [2]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [5]:
IMG_PATH="../facial_expressions/"

In [184]:
emotions=["anger", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

In [111]:
df=pd.read_csv(IMG_PATH+"icml_face_data.csv")

In [113]:
df=df.replace(to_replace="PrivateTest",value="Testing")
df=df.replace(to_replace="PublicTest",value="Training")

In [114]:
df

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...,...
35882,6,Testing,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
35883,3,Testing,178 174 172 173 181 188 191 194 196 199 200 20...
35884,0,Testing,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
35885,3,Testing,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...


In [115]:
data={"Training":[],"Testing":[]}

In [116]:
for i in range(len(df)):
    print("\r",i+1,end=" ")
    row=df.iloc[i]
    img=np.array(list(map(np.uint8,row[2].split()))).reshape(48,48)
    data[row[1]].append([row[0],ret_keypoints(img)])

 35887  4965      32797  

In [138]:
import pickle

In [139]:
with open("data.pkl","wb") as f:
    pickle.dump(data, f)

In [ ]:
# with open('data.pkl', 'rb') as f:
#     data = pickle.load(f)

In [140]:
face_dim=64

In [141]:
import matplotlib.pyplot as plt

In [142]:
def ret_keypoints(image):
    gray=image
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim))
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    param=[0,0,face_dim,face_dim]
    shape=predictor(gray,dlib.rectangle(*param))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(np.float32(shape.part(i).x))
        ylist.append(np.float32(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
#     plt.imshow(gray,cmap='gray')
#     plt.scatter(xlist,ylist, marker='.')
#     plt.show()
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim+1)/2

In [143]:
# s=np.arange(len(XXD))
# for i in range(5):
#     np.random.shuffle(s)
#     XXD=XXD[s]
#     YYD=YYD[s]

In [144]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [ ]:
model=Sequential()
model.add(Dense(1024,activation='tanh', input_shape=(68*2,)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(512,activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256,activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128,activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(len(emotions),activation='softmax'))

In [ ]:
model.summary()

In [213]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [188]:
XDB=np.vstack(np.asarray(data["Training"])[:,1])
YDY=np.asarray(data["Training"])[:,0]
YDB=np.zeros((len(YDY),len(emotions)))
for i in range(len(YDB)):
    YDB[i][YDY[i]]=1
XTB=np.vstack(np.asarray(data["Testing"])[:,1])
YTY=np.asarray(data["Testing"])[:,0]
YTB=np.zeros((len(YTY),len(emotions)))
for i in range(len(YTB)):
    YTB[i][YTY[i]]=1

In [315]:
model.fit(XDB,YDB,batch_size=64,epochs=100,validation_data=(XTB,YTB))

Train on 32298 samples, validate on 3589 samples
Epoch 1/100
32298/32298 [==============================] - 7s 214us/sample - loss: 1.2488 - accuracy: 0.5171 - val_loss: 1.4242 - val_accuracy: 0.4308
Epoch 2/100
32298/32298 [==============================] - 7s 204us/sample - loss: 1.2542 - accuracy: 0.5141 - val_loss: 1.3294 - val_accuracy: 0.4909
Epoch 3/100
32298/32298 [==============================] - 6s 194us/sample - loss: 1.2510 - accuracy: 0.5160 - val_loss: 1.3491 - val_accuracy: 0.4564
Epoch 4/100
32298/32298 [==============================] - 6s 194us/sample - loss: 1.2542 - accuracy: 0.5158 - val_loss: 1.4991 - val_accuracy: 0.4085
Epoch 5/100
32298/32298 [==============================] - 6s 193us/sample - loss: 1.2483 - accuracy: 0.5173 - val_loss: 1.3052 - val_accuracy: 0.4935
Epoch 6/100
32298/32298 [==============================] - 6s 195us/sample - loss: 1.2470 - accuracy: 0.5173 - val_loss: 1.3411 - val_accuracy: 0.4876
Epoch 7/100
32298/32298 [====================

KeyboardInterrupt: 

In [313]:
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# model.save_weights("model.h5")

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")

In [316]:
(model.predict(XTB).argmax(axis=1)==YTB.argmax(axis=1)).sum()/len(XTB)

0.4993034271384787

In [317]:
model.predict(XTB).argmax(axis=1)

array([6, 4, 4, ..., 4, 4, 4])

In [319]:
YTB.argmax(axis=1)

array([0, 5, 6, ..., 0, 3, 2])

In [288]:
from importlib import reload

In [290]:
import udp_streamer

In [291]:
reload(udp_streamer)

<module 'udp_streamer' from '/home/archer/machine_learning/expression_recognition/udp_streamer.py'>

In [292]:
from udp_streamer import *

In [302]:
handler = udp_handler()
handler.make_listener('0.0.0.0',5554)

In [267]:
from mtcnn import MTCNN
detector = MTCNN()

Using TensorFlow backend.


In [268]:
def face_rectangles(rgb):
    faces = detector.detect_faces(rgb)
    all_recs = []
    for x in faces:
        rec = x['box']
        all_recs.append(dlib.rectangle(left=rec[0], top=rec[1], right=rec[0]+rec[2], bottom=rec[1]+rec[3]))
    return all_recs

In [303]:
def get_img():
    global handler,img
    buffer = handler.get_data()
    if buffer is not None:
        try:
            npimg = np.frombuffer(buffer, dtype=np.uint8)
            img = cv2.imdecode(npimg, 1)
        except Exception as e:
            print(e)
    return img

In [322]:
# cam = cv2.VideoCapture(0)

while True:
#     ret, img = cam.read()
    img=get_img()
    try:
        grimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    except:
        continue
    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     faces = face_cascade.detectMultiScale(gimg, 1.3, 5)
    faces = face_rectangles(gimg)
#     for (x,y,w,h) in faces:
    for facrec in faces:
        x=facrec.left()
        y=facrec.top()
        w=facrec.right()-x
        h=facrec.bottom()-y
        landmarks=[]
        svx,svy=x,y
        gray=grimg[y:y+int(1.1*h),max(0,x-int(0.15*w)):x+int(1.15*w)]
        try:
            gray=cv2.resize(gray, (face_dim, face_dim))
        except:
            continue
        shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
        xlist=[]
        ylist=[]
        for i in range(68):
            xp=shape.part(i).x
            yp=shape.part(i).y
            cv2.circle(gray, (xp, yp), 2, (255, 255, 255), -1)
            xlist.append(float(xp))
            ylist.append(float(yp))
        cv2.imshow('gray',gray)
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        res=[]
        for (x,y) in zip(xcentral,ycentral):
            res.append(x)
            res.append(y)
        landmarks.append((np.asarray(res)/face_dim+1)/2)
        if len(landmarks)>0:
            y_out=model.predict(np.asarray(landmarks))
            res=np.argmax(y_out,axis=1)
            for r in res:
                cv2.putText(img,emotions[r],(svx,svy),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2,cv2.LINE_AA)
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xff == 27:
        break

cam.release()
cv2.destroyAllWindows()

In [309]:
cam.release()
cv2.destroyAllWindows()